In [2]:
import os
import re

def get_mesa_params_from_inlist(inlist_file_path):
    """
    Kinyeri az initial_mass és Zbase paramétereket egy MESA inlist fájlból.
    """
    mass = None
    z = None

    try:
        with open(inlist_file_path, 'r') as f:
            content = f.read()

            # Reguláris kifejezésekkel keressük a paramétereket
            mass_match = re.search(r'^\s*(?:initial_mass|new_mass)\s*=\s*(\d+\.?\d*(?:e[+\-]?\d+)?)\s*$', content, re.MULTILINE | re.IGNORECASE)
            z_match = re.search(r'^\s*(?:Zbase|new_z)\s*=\s*(\d+\.?\d*(?:e[+\-]?\d+)?)\s*$', content, re.MULTILINE | re.IGNORECASE)

            if mass_match:
                mass = float(mass_match.group(1))
            if z_match:
                z = float(z_match.group(1))

    except Exception as e:
        # Ezt az üzenetet akkor látod, ha egy inlist fájl olvasásakor hiba történik (pl. jogosultság, vagy sérült fájl)
        print(f"Figyelem: Hiba történt az inlist ({inlist_file_path}) olvasásakor: {e}")

    return mass, z

def get_params_for_single_mesa_run_from_inlist(base_run_directory):
    """
    Kinyeri a tömeg (M) és metallicitás (Z) paramétereket egyetlen MESA futásból.
    Feltételezi, hogy a base_run_directory tartalmazza az 'inlist_project' fájlt.

    Args:
        base_run_directory (str): Annak a MESA futásnak a gyökérkönyvtára,
                                  ami az 'inlist_project' fájlt tartalmazza.

    Returns:
        tuple: (mass, z) ha sikeres, különben (None, None).
    """
    if not os.path.isdir(base_run_directory):
        return None, None

    inlist_file_path = os.path.join(base_run_directory, "inlist_project")

    if not os.path.exists(inlist_file_path):
        return None, None

    mass, z = get_mesa_params_from_inlist(inlist_file_path)

    if mass is not None and z is not None:
        return mass, z
    else:
        return None, None

def analyze_mesa_grid_directory(grid_base_path):
    """
    Bejárja a megadott gyökérkönyvtárat, és minden alkönyvtárban megpróbálja
    kinyerni a MESA futások tömeg (M) és metallicitás (Z) paramétereit
    az 'inlist_project' fájlokból.

    Args:
        grid_base_path (str): A MESA futások gyökérkönyvtára, pl. '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE'.

    Returns:
        list: Egy lista szótárakból, ahol minden szótár egy futás adatait tartalmazza:
              {'path': '/elérési/út', 'mass': 1.0, 'z': 0.02}
    """
    found_runs = []
    print(f"MESA grid elemzés indítása a következő gyökérkönyvtárban: '{grid_base_path}'")

    if not os.path.isdir(grid_base_path):
        print(f"Hiba: A megadott gyökérkönyvtár '{grid_base_path}' nem létezik vagy nem mappa.")
        return []

    # os.walk bejárja a könyvtárat rekurzívan
    for root, dirs, files in os.walk(grid_base_path):
        # Ha a jelenlegi 'root' (alkönyvtár) tartalmaz 'inlist_project' fájlt,
        # akkor feltételezzük, hogy ez egy MESA futás könyvtára.
        if "inlist_project" in files:
            mass, z = get_params_for_single_mesa_run_from_inlist(root)
            if mass is not None and z is not None:
                found_runs.append({
                    'path': root,
                    'mass': mass,
                    'z': z
                })
                # Ez a print sor segít látni, hol tart a keresés
                print(f"  Talált futás: M={mass}, Z={z} in '{root}'")

    print(f"\nElemzés befejezve. Összesen {len(found_runs)} MESA futás került beazonosításra.")
    return found_runs

# --- Példa a használatra ---
if __name__ == "__main__":
    # >>>>>> ITT ADD MEG A MESA GRID GYÖKÉRKÖNYVTÁRÁT <<<<<<
    # A te példád alapján ez lehet:
    # mesa_grid_root_directory = "/home/tnehezd/workdir/mesa-r23.05.1/STRANGE"
    # VAGY a specifikusabb mappa, ha abban vannak a futások:
    mesa_grid_root_directory = "/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid"

    # Fontos: Győződj meg róla, hogy a fenti útvonal helyes a rendszereden!
    # Ha van egy teszt mappád (pl. amiben az "M_1.0_Z_0.02" könyvtárak vannak),
    # akkor azt add meg itt!

    print(f"\n--- MESA grid elemzés indítása: '{mesa_grid_root_directory}' ---")
    real_mesa_runs = analyze_mesa_grid_directory(mesa_grid_root_directory)

    print("\n--- Összesített MESA futás paraméterek ---")
    if real_mesa_runs:
        for run in real_mesa_runs:
            print(f"  Mappa: {run['path']}, Tömeg: {run['mass']}, Z: {run['z']}")

        # Ezen a ponton a 'real_mesa_runs' lista tartalmazza az összes talált futás adatait.
        # Itt folytathatod a pipeline-od azon részével, ami az adott futás (M, Z) adatait feldolgozza.
        # Például:
        # for run_data in real_mesa_runs:
        #     run_path = run_data['path']
        #     mass_val = run_data['mass']
        #     z_val = run_data['z']
        #
        #     history_file = os.path.join(run_path, "history.data")
        #     if os.path.exists(history_file):
        #         print(f"  Feldolgozom a history.data fájlt innen: {history_file}")
        #         # Itt olvashatod be pl. pandas-szal:
        #         # import pandas as pd
        #         # df = pd.read_csv(history_file, delim_whitespace=True, skiprows=5) # Gyakori formátum MESA history.data esetén
        #         # print(f"    Első sor a history.data-ból:\n{df.head(1)}")
        #     else:
        #         print(f"  history.data nem található: {history_file}")
        #
        print("\nAz elemzés és további feldolgozás folytatódhat az összes kinyert paraméterrel.")
    else:
        print("Nem találtunk érvényes MESA futásokat a megadott gyökérkönyvtárban.")


--- MESA grid elemzés indítása: '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid' ---
MESA grid elemzés indítása a következő gyökérkönyvtárban: '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid'
  Talált futás: M=11.0, Z=0.0015 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid'
  Talált futás: M=12.5, Z=0.0165 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_12.5MSUN_z0.0165'
  Talált futás: M=7.0, Z=0.013 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_7.0MSUN_z0.0130'
  Talált futás: M=8.5, Z=0.003 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_8.5MSUN_z0.0030'
  Talált futás: M=7.5, Z=0.019 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_7.5MSUN_z0.0190'
  Talált futás: M=5.0, Z=0.0155 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_5.0MSUN_z0.0155'
  Talált futás: M=5.0, Z=0.0025 

  Talált futás: M=3.5, Z=0.007 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_3.5MSUN_z0.0070'
  Talált futás: M=7.0, Z=0.018 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_7.0MSUN_z0.0180'
  Talált futás: M=7.0, Z=0.0045 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_7.0MSUN_z0.0045'
  Talált futás: M=4.5, Z=0.0185 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_4.5MSUN_z0.0185'
  Talált futás: M=6.0, Z=0.0115 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_6.0MSUN_z0.0115'
  Talált futás: M=13.0, Z=0.0135 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_13.0MSUN_z0.0135'
  Talált futás: M=5.5, Z=0.0175 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_5.5MSUN_z0.0175'
  Talált futás: M=13.0, Z=0.0025 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/na

  Talált futás: M=9.0, Z=0.0185 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_9.0MSUN_z0.0185'
  Talált futás: M=5.0, Z=0.0135 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_5.0MSUN_z0.0135'
  Talált futás: M=5.0, Z=0.0145 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_5.0MSUN_z0.0145'
  Talált futás: M=5.0, Z=0.0075 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_5.0MSUN_z0.0075'
  Talált futás: M=9.0, Z=0.0075 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_9.0MSUN_z0.0075'
  Talált futás: M=5.5, Z=0.004 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_5.5MSUN_z0.0040'
  Talált futás: M=15.0, Z=0.0045 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_15.0MSUN_z0.0045'
  Talált futás: M=13.5, Z=0.0015 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/n

  Talált futás: M=5.5, Z=0.003 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_5.5MSUN_z0.0030'
  Talált futás: M=10.5, Z=0.0025 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_10.5MSUN_z0.0025'
  Talált futás: M=9.0, Z=0.0115 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_9.0MSUN_z0.0115'
  Talált futás: M=6.5, Z=0.02 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_6.5MSUN_z0.0200'
  Talált futás: M=13.5, Z=0.015 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_13.5MSUN_z0.0150'
  Talált futás: M=3.0, Z=0.0095 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_3.0MSUN_z0.0095'
  Talált futás: M=9.0, Z=0.006 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_9.0MSUN_z0.0060'
  Talált futás: M=8.5, Z=0.0145 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_

  Talált futás: M=10.0, Z=0.0115 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_10.0MSUN_z0.0115'
  Talált futás: M=9.5, Z=0.0055 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_9.5MSUN_z0.0055'
  Talált futás: M=4.5, Z=0.0105 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_4.5MSUN_z0.0105'
  Talált futás: M=13.5, Z=0.006 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_13.5MSUN_z0.0060'
  Talált futás: M=13.0, Z=0.015 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_13.0MSUN_z0.0150'
  Talált futás: M=14.0, Z=0.0045 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_14.0MSUN_z0.0045'
  Talált futás: M=14.0, Z=0.003 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_14.0MSUN_z0.0030'
  Talált futás: M=12.5, Z=0.012 in '/home/tnehezd/workdir/mesa-r23.05.1/STRA

...

  Talált futás: M=13.0, Z=0.0055 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_13.0MSUN_z0.0055'
  Talált futás: M=13.0, Z=0.0115 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_13.0MSUN_z0.0115'
  Talált futás: M=10.5, Z=0.013 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_10.5MSUN_z0.0130'
  Talált futás: M=13.0, Z=0.0105 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_13.0MSUN_z0.0105'
  Talált futás: M=2.5, Z=0.0065 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_2.5MSUN_z0.0065'
  Talált futás: M=12.5, Z=0.0095 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_12.5MSUN_z0.0095'
  Talált futás: M=4.0, Z=0.0155 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_4.0MSUN_z0.0155'
  Talált futás: M=13.0, Z=0.0085 in '/home/tnehezd/workdir/mesa-r23.05.1/S

  Talált futás: M=11.5, Z=0.0195 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_11.5MSUN_z0.0195'
  Talált futás: M=14.5, Z=0.0175 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_14.5MSUN_z0.0175'
  Talált futás: M=8.0, Z=0.0135 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_8.0MSUN_z0.0135'
  Talált futás: M=10.0, Z=0.002 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_10.0MSUN_z0.0020'
  Talált futás: M=4.0, Z=0.017 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_4.0MSUN_z0.0170'
  Talált futás: M=12.5, Z=0.0105 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_12.5MSUN_z0.0105'
  Talált futás: M=11.5, Z=0.012 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_11.5MSUN_z0.0120'
  Talált futás: M=7.0, Z=0.012 in '/home/tnehezd/workdir/mesa-r23.05.1/STRAN

  Talált futás: M=8.5, Z=0.0135 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_8.5MSUN_z0.0135'
  Talált futás: M=7.5, Z=0.01 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_7.5MSUN_z0.0100'
  Talált futás: M=11.5, Z=0.006 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_11.5MSUN_z0.0060'
  Talált futás: M=12.0, Z=0.0045 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_12.0MSUN_z0.0045'
  Talált futás: M=3.5, Z=0.005 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_3.5MSUN_z0.0050'
  Talált futás: M=9.5, Z=0.014 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_9.5MSUN_z0.0140'
  Talált futás: M=14.0, Z=0.012 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_14.0MSUN_z0.0120'
  Talált futás: M=14.5, Z=0.0045 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/na

  Talált futás: M=8.0, Z=0.0035 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_8.0MSUN_z0.0035'
  Talált futás: M=2.0, Z=0.002 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_2.0MSUN_z0.0020'
  Talált futás: M=13.5, Z=0.0025 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_13.5MSUN_z0.0025'
  Talált futás: M=11.0, Z=0.02 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_11.0MSUN_z0.0200'
  Talált futás: M=14.0, Z=0.0105 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_14.0MSUN_z0.0105'
  Talált futás: M=9.0, Z=0.0065 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_9.0MSUN_z0.0065'
  Talált futás: M=11.5, Z=0.0185 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_11.5MSUN_z0.0185'
  Talált futás: M=3.0, Z=0.0035 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANG

  Talált futás: M=14.0, Z=0.009 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_14.0MSUN_z0.0090'
  Talált futás: M=5.5, Z=0.0055 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_5.5MSUN_z0.0055'
  Talált futás: M=4.0, Z=0.002 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_4.0MSUN_z0.0020'
  Talált futás: M=4.5, Z=0.02 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_4.5MSUN_z0.0200'
  Talált futás: M=15.0, Z=0.005 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_15.0MSUN_z0.0050'
  Talált futás: M=15.0, Z=0.007 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_15.0MSUN_z0.0070'
  Talált futás: M=3.0, Z=0.011 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid/run_nad_convos_mid_3.0MSUN_z0.0110'
  Talált futás: M=11.0, Z=0.011 in '/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_

In [3]:
# In your Jupyter Notebook (.ipynb file)

# Import the main function from your package
from mesa_tools.grid_analyzer import analyze_mesa_grid_directory

# Configure your MESA grid root directory
# IMPORTANT: Replace this with the actual path to your MESA grid!
my_mesa_grid_path = "/home/tnehezd/workdir/mesa-r23.05.1/STRANGE/nad_convos_mid" 

print(f"Starting MESA grid analysis on: '{my_mesa_grid_path}'")

# Run the analysis
found_mesa_runs = analyze_mesa_grid_directory(my_mesa_grid_path)

# You can now work with 'found_mesa_runs' in your notebook
# For example, convert it to a pandas DataFrame for easier manipulation:
# import pandas as pd
# df_runs = pd.DataFrame(found_mesa_runs)
# print("\nDataFrame of found MESA runs:")
# display(df_runs.head()) # Use display() in Jupyter for nice output

# ... and continue with your data analysis, plotting, etc.

ModuleNotFoundError: No module named 'mesa_tools.inlist_parser'